In [1]:
import random
import pprint
import time
import copy
import uuid


In [2]:
# import sys
# old_stdout = sys.stdout

# log_file = open("message.log", "w")

# sys.stdout = log_file


In [3]:
weights = [0,]
for x in range(1001):
    w = random.randint(1,100)
    weights.append(w)

In [4]:
itemsInfo = {}
binsInfo = {}
probabilites = [0,]
# weights = [0,78, 88, 52, 77, 17, 85, 39, 95, 37, 58, 39, 21,53, 10, 69, 81, 28, 32, 48, 56, 88, 59, 29, 97, 43]
capacitites = [0,60, 100, 150]
costs = [0, 60, 100, 150]


for x in range(1,len(capacitites)):
    probabilites.append(capacitites[x]/costs[x])

for x in range(1, 1001):
    itemsInfo[f"item{x}"] = weights[x]

for x in range(1, 4):
    binsInfo[x] = {
        "cost": capacitites[x],
        "capacity": costs[x],
        "probability":  probabilites[x]
    }

print(itemsInfo,binsInfo)


{'item1': 73, 'item2': 53, 'item3': 68, 'item4': 11, 'item5': 78, 'item6': 18, 'item7': 39, 'item8': 28, 'item9': 65, 'item10': 19, 'item11': 5, 'item12': 15, 'item13': 52, 'item14': 69, 'item15': 95, 'item16': 45, 'item17': 54, 'item18': 48, 'item19': 97, 'item20': 92, 'item21': 23, 'item22': 78, 'item23': 53, 'item24': 41, 'item25': 14, 'item26': 52, 'item27': 14, 'item28': 89, 'item29': 24, 'item30': 6, 'item31': 21, 'item32': 96, 'item33': 21, 'item34': 65, 'item35': 40, 'item36': 62, 'item37': 31, 'item38': 25, 'item39': 20, 'item40': 37, 'item41': 69, 'item42': 48, 'item43': 100, 'item44': 87, 'item45': 90, 'item46': 52, 'item47': 99, 'item48': 77, 'item49': 21, 'item50': 77, 'item51': 90, 'item52': 31, 'item53': 25, 'item54': 53, 'item55': 5, 'item56': 46, 'item57': 73, 'item58': 95, 'item59': 47, 'item60': 51, 'item61': 41, 'item62': 35, 'item63': 52, 'item64': 86, 'item65': 86, 'item66': 98, 'item67': 57, 'item68': 53, 'item69': 69, 'item70': 27, 'item71': 12, 'item72': 31, 'i

In [5]:
def GenerateInitialSolution() :
    solution = {}
    s = 1
    for id,w in itemsInfo.items():
        newId = uuid.uuid4()
        validBinTypes = list(filter(lambda bin : binsInfo[bin]["capacity"] > w,binsInfo ))
        validProbabilites = tuple(map(lambda bin: binsInfo[bin]["probability"], validBinTypes))
        selectedBin = random.choices(validBinTypes,weights=validProbabilites,k=1)[0]
        remainingCapacity = binsInfo[selectedBin]["capacity"] - itemsInfo[id] 
        solution[newId] = {"bin_type": selectedBin, "item_ids": [id], "remaining_capacity": remainingCapacity}
        s+= 1
    return solution

In [6]:
def Cost(x : dict):
    totalCost = 0
    for info in x.values() :
        totalCost += binsInfo[info["bin_type"]]["cost"]
    return totalCost

In [7]:
def ChooseNeightbourhood(c1,c2) :
    k = random.choices([1,2],weights=(c1,c2),k=1)[0]
    return k


In [8]:
def RemainingCapacity(x,id):
    capacity = binsInfo[x[id]["bin_type"]]["capacity"]
    weight = 0
    if len(x[id]["item_ids"]) == 0 :
        return capacity
    for itemId in x[id]["item_ids"]:
        weight += itemsInfo[itemId]
    return capacity - weight

In [9]:
def GetWeight(id):
    return itemsInfo[id]

def UnpackRepack(x : dict,binId,newCapacity,newBinType) :
    
    cap = newCapacity
    unpackedItems = []

    #Unpacking 
    while cap < 0 :
        unpackedItem = random.choice(x[binId]["item_ids"])
        unpackedItems.append(unpackedItem)
        x[binId]["item_ids"].remove(unpackedItem)
        cap += itemsInfo[unpackedItem]
    x[binId]["remaining_capacity"] = cap

    if (len(x[binId]["item_ids"]) == 0) :
        del x[binId]

    #Repacking
    unpackedItems.sort(key=GetWeight,reverse=True)
    for id in unpackedItems:
        validBins = list(filter(lambda bin: x[bin]["remaining_capacity"] > itemsInfo[id], x.keys()))
        if (len(validBins) == 0):
            newId = uuid.uuid4()
            validBinTypes = list(filter(lambda bin: binsInfo[bin]["capacity"] > itemsInfo[id], binsInfo))
            validProbabilites = tuple(map(lambda bin: binsInfo[bin]["probability"], validBinTypes))
            selectedBin = random.choices(validBinTypes, weights=validProbabilites, k=1)[0]
            remainingCapacity = binsInfo[selectedBin]["capacity"] - itemsInfo[id]
            x[newId] = {"bin_type": selectedBin, "item_ids": [id], "remaining_capacity": remainingCapacity}
        else :
            validProbabilites = tuple(map(lambda bin: x[bin]["remaining_capacity"], validBins))
            # print(f"Item : {id} {itemsInfo[id]} | Bins : {validBins} {validProbabilites}")
            selectedBin = random.choices(validBins, weights=validProbabilites, k=1)[0]
            x[selectedBin]["item_ids"].append(id)
            x[selectedBin]["remaining_capacity"] = RemainingCapacity(x, selectedBin)
            # print(f"{selectedBin} {x[selectedBin]['remaining_capacity']} ")


    return x


In [10]:
def ValidateSolution(x : dict):
    for id,info in x.items():
        print(f"Bin type : {info['bin_type']} | Item ID : {info['item_ids']} | Remaining Capacity :{info['remaining_capacity']}")
        weight = 0
        for itemId in info['item_ids'] :
            weight += itemsInfo[itemId]
        capacity = binsInfo[info['bin_type']]['capacity']
        # print(f"Calculated Weight {weight} | Calculated Capacity {capacity} | Calculated Remaining Capacity : {capacity - weight}")
        print("")

In [11]:
def N1(x: dict):
    binIds = list(x.keys())
    selected_length = int(len(binIds)*0.15)
    bin_samples = random.sample(binIds, selected_length)
    for binId in bin_samples:
        newBinType = random.choice(
            [i for i in range(1, 4) if i != x[binId]["bin_type"]])
        oldCapacity = RemainingCapacity(x, binId)
        oldBinType = x[binId]["bin_type"]
        x[binId]["bin_type"] = newBinType
        newCapacity = RemainingCapacity(x, binId)
        x[binId]["remaining_capacity"] = newCapacity
        # print(f"{binId} | New : {newBinType} {newCapacity} | Old : {oldBinType} {oldCapacity}")

        if newCapacity < 0:
            x = UnpackRepack(x, binId, newCapacity, newBinType)
    return x


In [12]:
def GetBin(x,itemId):
    for id,info in x.items():
        if itemId in info["item_ids"]:
            info["bin_id"] = id
            return info

In [13]:
def GetCapacity(bin):
    return bin["remaining_capacity"]


def N4(x : dict):
    itemStuff = {}
    for id,info in x.items() :
        for itemId in info["item_ids"]:
            itemStuff[itemId] = {"bin_id": id, "remaining_capacity": info["remaining_capacity"]}
    for id in itemsInfo :
        binA = GetBin(x,id) 
        validBins = list(filter(lambda bin: x[bin]["remaining_capacity"] > itemsInfo[id] and x[bin]["remaining_capacity"] - itemsInfo[id] < binA["remaining_capacity"] and bin != binA["bin_id"], x.keys()))
        validBinInfo = list(map(lambda bin: {"remaining_capacity": x[bin]["remaining_capacity"], "bin_id": bin}, validBins))
        validBinInfo.sort(key=GetCapacity)
        if len(validBinInfo) == 0 :
            continue 
        binB = validBinInfo[0]
        # print(itemsInfo[id],binA, binB)

        x[binA["bin_id"]]["item_ids"].remove(id)
        x[binA["bin_id"]]["remaining_capacity"] = RemainingCapacity(x, binA["bin_id"])
        if (len(x[binA["bin_id"]]["item_ids"]) == 0):
            del x[binA["bin_id"]]

        x[binB["bin_id"]]["item_ids"].append(id)
        x[binB["bin_id"]]["remaining_capacity"] = RemainingCapacity(x, binB["bin_id"])
        
        # print("After Shift :")
        # print(x[binA["bin_id"]], x[binB["bin_id"]])
        # print()
    return x

def N5(x : dict):
    itemStuff = {}
    for id,info in x.items() :
        for itemId in info["item_ids"]:
            itemStuff[itemId] = {"bin_id": id, "remaining_capacity": info["remaining_capacity"]}
    for itemId in itemsInfo:
        binA = GetBin(x,itemId)
        
        aRemainingCApacity = binA["remaining_capacity"]
        validIds = {}
        for id,info in x.items():
            itemIds = info["item_ids"]
            bRemainingCapacity = info["remaining_capacity"]
            itemIds.sort(key=GetWeight,reverse=True)
            swappedId = ""
            for sId in itemIds:
                # print(itemsInfo[sId], bRemainingCapacity + itemsInfo[sId], itemsInfo[itemId])
                if bRemainingCapacity + itemsInfo[sId] > itemsInfo[itemId] and itemsInfo[itemId] > itemsInfo[sId] and bRemainingCapacity + itemsInfo[sId] - itemsInfo[itemId] < aRemainingCApacity:
                    swappedId = sId
            if swappedId == "" :
                continue
            # print(
            #     f"SWAP : {swappedId} {itemsInfo[swappedId]} | Item : {itemId} {itemsInfo[itemId]}   ")
            validIds[swappedId] =  bRemainingCapacity +itemsInfo[swappedId] - itemsInfo[itemId]
        # print("..................")
        finalId = ""
        for ab in validIds.keys() :
            if finalId == "":
                finalId = ab
                continue
            if validIds[finalId] > validIds[ab]:
                finalId = ab
        # print(validIds,finalId)
        if (finalId == ""):
            continue
        binB = GetBin(x,finalId)
        # print(f"Swapping {x[binA['bin_id']]} and {x[binB['bin_id']]}")
        # print(f"binA item : {itemId} {itemsInfo[itemId]} | binB item : {finalId} {itemsInfo[finalId]}")
        # print(f"binA capacity : {binA['remaining_capacity']} binB capacity : {binB['remaining_capacity']}  ")

        x[binA["bin_id"]]["item_ids"].remove(itemId)
        x[binA["bin_id"]]["item_ids"].append(finalId)
        x[binA["bin_id"]]["remaining_capacity"] = RemainingCapacity(x, binA["bin_id"])
        
        x[binB["bin_id"]]["item_ids"].remove(finalId)
        x[binB["bin_id"]]["item_ids"].append(itemId)
        x[binB["bin_id"]]["remaining_capacity"] = RemainingCapacity(x, binB["bin_id"])
        # print(f"new binA capacity : {x[binA['bin_id']]['remaining_capacity']} new binB capacity : {x[binB['bin_id']]['remaining_capacity']}")
    return x

def ChangeBinType(x):
    for id,info in x.items():
        remainingCapacity = info["remaining_capacity"]
        binSize = binsInfo[info["bin_type"]]["capacity"]
        for type,binInfo in binsInfo.items():
            if binSize - remainingCapacity < binInfo["capacity"] and binSize > binInfo["capacity"]:
                x[id]["bin_type"] = type
                x[id]["remaining_capacity"] = RemainingCapacity(x,id)
                # print(type, binInfo["capacity"], remainingCapacity, binSize, RemainingCapacity(x, id), binInfo["capacity"] - binSize + remainingCapacity)
    return x

def LocalSearch(x1):
    x3 = N4(x1)
    # x4 = N5(x3)
    # x5 = N6(x4)

    return x3

def N6(x):
    high = 0
    removeId = ""
    for id,info in x.items():
        if info["remaining_capacity"] > high:
            removeId = id
            high = info["remaining_capacity"]
    itemIds = x[removeId]["item_ids"]

    for itemId in itemIds:
        low = 9999
        addId = ""
        weight = itemsInfo[itemId]
        for id,info in x.items():
            if (info["remaining_capacity"] < low and info["remaining_capacity"] > weight):
                addId = id
                low = info["remaining_capacity"]
        
        if (addId == ""):
            continue
        # print("Doing stuff")
        x[removeId]["item_ids"].remove(itemId)
        x[removeId]["remaining_capacity"] = RemainingCapacity(x, removeId)

        x[addId]["item_ids"].append(itemId)
        x[addId]["remaining_capacity"] = RemainingCapacity(x, addId)
    
    if len(x[removeId]["item_ids"]) == 0:
        del x[removeId]

    return x
    
        


def LocalSearchv2(x1):
    x3 = N4(x1)
    x4 = N5(x3)
    x5 = N6(x4)
    x6 = ChangeBinType(x4)
    return x6


In [14]:
def N2(x):
    binIds = list(x.keys())
    selected_length = int(len(binIds)*0.05)
    bin_samples = random.sample(binIds, selected_length)
    for binId in bin_samples:
        itemIds = x[binId]["item_ids"]
        del x[binId]
        for id in itemIds:
            newId = uuid.uuid4()
            w = itemsInfo[id]
            validBinTypes = list(filter(lambda bin: binsInfo[bin]["capacity"] > w, binsInfo))
            validProbabilites = tuple(map(lambda bin: binsInfo[bin]["probability"], validBinTypes))
            selectedBin = random.choices(validBinTypes, weights=validProbabilites, k=1)[0]
            remainingCapacity = binsInfo[selectedBin]["capacity"] - itemsInfo[id]
            x[newId] = {"bin_type": selectedBin, "item_ids": [id], "remaining_capacity": remainingCapacity}
    return x


In [15]:
def Shaking(x, k):
    if (k == 1):
        shook = N1(x)
    if (k == 2):
        shook = N2(x)
    else:
        return x
    return shook

In [16]:
def N1N2LS():
    x = GenerateInitialSolution()
    # print(f"Initial Bins Used {len(x.keys())}")
    # print(f"Initial Cost {Cost(x)}\n")
    # ValidateSolution(x)
    finish = False
    count = 0
    c1, c2, c3 = 1, 1, 1
    c1Total, c2Total, c3Total = 1,1,1
    total = 0
    startTime = time.time()
    while not finish:
        k = ChooseNeightbourhood(c1/c1Total,c2/c2Total)
        if k == 1:
            c1Total +=1
        if k == 2:
            c2Total +=1
        oldX = copy.deepcopy(x)
        x1 = Shaking(x, 1)
        x2 = LocalSearchv2(x1)
        # print(f"Old Cost : {Cost(oldX)} New Cost : {Cost(x2)}")
        # print(f"Chosen Neighbourhood {k}")
        if (Cost(x2) < Cost(oldX)):
            if (k== 1):
                c1 += 1
            if (k == 2):
                c2 += 1
            updatedTime = time.time()
            x = x2
        else:
            x = oldX
        if (time.time() - startTime > 50):
            finish = True
    totalTime = updatedTime - startTime
    totalBins = len(x.keys())
    print(f"Total Bins Used {totalBins}")
    print(f"Final Optimised Cost {Cost(x)}")
    print(f"Last updated time : {totalTime}")
    print("")
    ValidateSolution(x)

    return totalBins, Cost(x), totalTime


N1N2LS_BINS, N1N2LS_COST, N1N2LS_TIME = N1N2LS()


Total Bins Used 427
Final Optimised Cost 50810
Last updated time : 24.185479640960693

Bin type : 3 | Item ID : ['item15', 'item17'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item24', 'item936', 'item6'] | Remaining Capacity :1

Bin type : 3 | Item ID : ['item32', 'item2'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item894'] | Remaining Capacity :1

Bin type : 1 | Item ID : ['item52', 'item8'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item23', 'item73'] | Remaining Capacity :1

Bin type : 3 | Item ID : ['item209', 'item216', 'item540', 'item425'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item803', 'item39', 'item78'] | Remaining Capacity :1

Bin type : 1 | Item ID : ['item468', 'item371'] | Remaining Capacity :3

Bin type : 3 | Item ID : ['item91', 'item80'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item94', 'item96'] | Remaining Capacity :1

Bin type : 3 | Item ID : ['item403', 'item948', 'item792'] | Remaining Capacity :1

Bin type : 2

In [17]:
def N1N4N5N6():
    x = GenerateInitialSolution()
    # print(f"Initial Bins Used {len(x.keys())}")
    # print(f"Initial Cost {Cost(x)}\n")
    # ValidateSolution(x)
    finish = False
    count = 0
    c1, c2, c3 = 0, 1, 1
    startTime = time.time()
    while not finish:
        #     # k = ChooseNeightbourhood(c1,c2,c3)
        oldX = copy.deepcopy(x)
        count += 1
        x1 = Shaking(x, 1)
        x2 = LocalSearchv2(x1)
        # print(f"Old Cost : {Cost(oldX)} New Cost : {Cost(x2)}")
        if (Cost(x2) < Cost(oldX)):
            c1 += 1
            updatedTime = time.time()
            x = x2
        else:
            x = oldX
        if (time.time() - startTime > 50):
            finish = True
    totalTime = updatedTime - startTime
    totalBins = len(x.keys())
    print(f"Total Bins Used {totalBins}")
    print(f"Final Optimised Cost {Cost(x)}")
    print(f"Last updated time : {totalTime}")
    print(c1)
    print("")
    ValidateSolution(x)

    return totalBins, Cost(x), totalTime

N1N4N5N6_BINS,N1N4N5N6_COST,N1N4N5N6_TIME = N1N4N5N6()

Total Bins Used 460
Final Optimised Cost 50870
Last updated time : 16.604002952575684
4

Bin type : 2 | Item ID : ['item17', 'item16'] | Remaining Capacity :1

Bin type : 1 | Item ID : ['item2', 'item30'] | Remaining Capacity :1

Bin type : 3 | Item ID : ['item32', 'item23'] | Remaining Capacity :1

Bin type : 1 | Item ID : ['item37', 'item8'] | Remaining Capacity :1

Bin type : 3 | Item ID : ['item5', 'item175'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item26', 'item59'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item68', 'item56'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item44', 'item71'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item76', 'item25'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item77', 'item62'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item22', 'item88'] | Remaining Capacity :1

Bin type : 3 | Item ID : ['item15', 'item93'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item95', 'item40'] | Remaining Cap

In [18]:
def N1N4():
    x = GenerateInitialSolution()
    # print(f"Initial Bins Used {len(x.keys())}")
    # print(f"Initial Cost {Cost(x)}\n")
    # ValidateSolution(x)
    finish = False
    count = 0
    c1, c2, c3 = 0, 1, 1
    startTime = time.time()
    while not finish:
        #     # k = ChooseNeightbourhood(c1,c2,c3)
        oldX = copy.deepcopy(x)
        count += 1
        x1 = Shaking(x, 1)
        x2 = LocalSearch(x1)
        # print(f"Old Cost : {Cost(oldX)} New Cost : {Cost(x2)}")
        if (Cost(x2) < Cost(oldX)):
            c1 += 1
            updatedTime = time.time()
            x = x2
        else:
            x = oldX
        if (time.time() - startTime > 50):
            finish = True
    totalTime = updatedTime - startTime
    totalBins = len(x.keys())
    print(f"Total Bins Used {totalBins}")
    print(f"Final Optimised Cost {Cost(x)}")
    print(f"Last updated time : {totalTime}")
    print(c1)
    print("")
    ValidateSolution(x)

    return totalBins,Cost(x),totalTime

N1N4_BINS,N1N4_COST,N1N4_TIME = N1N4()

Total Bins Used 454
Final Optimised Cost 50970
Last updated time : 39.85288858413696
6

Bin type : 3 | Item ID : ['item19', 'item63'] | Remaining Capacity :1

Bin type : 1 | Item ID : ['item35', 'item10'] | Remaining Capacity :1

Bin type : 1 | Item ID : ['item37', 'item8'] | Remaining Capacity :1

Bin type : 1 | Item ID : ['item42', 'item4'] | Remaining Capacity :1

Bin type : 3 | Item ID : ['item43', 'item734'] | Remaining Capacity :4

Bin type : 2 | Item ID : ['item49', 'item22'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item52', 'item3'] | Remaining Capacity :1

Bin type : 1 | Item ID : ['item68', 'item30'] | Remaining Capacity :1

Bin type : 2 | Item ID : ['item923'] | Remaining Capacity :4

Bin type : 1 | Item ID : ['item71', 'item59'] | Remaining Capacity :1

Bin type : 3 | Item ID : ['item78', 'item249', 'item14'] | Remaining Capacity :1

Bin type : 3 | Item ID : ['item97', 'item84'] | Remaining Capacity :1

Bin type : 1 | Item ID : ['item101', 'item93'] | Remaining C

In [19]:
def OnlyN1():
    # t0 = time.time()
    x = GenerateInitialSolution()
    # print(f"Initial Bins Used {len(x.keys())}")
    # print(f"Initial Cost {Cost(x)}\n")
    # ValidateSolution(x)
    
    finish = False
    count = 0
    c1,c2,c3 = 0,1,1
    startTime = time.time()
    while not finish:
    #     # k = ChooseNeightbourhood(c1,c2,c3)
        oldX = copy.deepcopy(x)
        count +=1
        x1 = Shaking(x,1)
        # print(f"Old Cost : {Cost(oldX)} New Cost : {Cost(x1)}")
        if (Cost(x1) < Cost(oldX)):
            c1+=1
            updatedTime = time.time()
            x = x1
        else :
            x = oldX
        if (time.time() - startTime > 50):
            finish = True
    totalTime = updatedTime - startTime
    totalBins = len(x.keys())
    print(f"Total Bins Used {totalBins}")
    print(f"Final Optimised Cost {Cost(x)}")
    print(f"Last updated time : {totalTime}")
    print(c1)
    print("")
    ValidateSolution(x)

    return totalBins, Cost(x), totalTime

N1_BINS,N1_COST,N1_TIME = OnlyN1()

Total Bins Used 517
Final Optimised Cost 56910
Last updated time : 47.25099301338196
63

Bin type : 1 | Item ID : ['item296'] | Remaining Capacity :19

Bin type : 2 | Item ID : ['item10', 'item49', 'item664'] | Remaining Capacity :45

Bin type : 2 | Item ID : ['item654', 'item268'] | Remaining Capacity :8

Bin type : 2 | Item ID : ['item784'] | Remaining Capacity :0

Bin type : 3 | Item ID : ['item21', 'item828'] | Remaining Capacity :90

Bin type : 2 | Item ID : ['item23', 'item39'] | Remaining Capacity :27

Bin type : 2 | Item ID : ['item25', 'item183', 'item636'] | Remaining Capacity :4

Bin type : 1 | Item ID : ['item539'] | Remaining Capacity :1

Bin type : 3 | Item ID : ['item35', 'item743'] | Remaining Capacity :55

Bin type : 2 | Item ID : ['item358', 'item491'] | Remaining Capacity :9

Bin type : 1 | Item ID : ['item46'] | Remaining Capacity :8

Bin type : 2 | Item ID : ['item278', 'item283', 'item626'] | Remaining Capacity :41

Bin type : 3 | Item ID : ['item364', 'item178', 

In [20]:
print(f"N1 | Cost = {N1_COST} | Bins = {N1_BINS} | Time = {N1_TIME}")
print(f"N1 and N4 | Cost = {N1N4_COST} | Bins = {N1N4_BINS} | Time ={N1N4_TIME}")
print(f"N1 and LS | Cost = {N1N4N5N6_COST} | Bins = {N1N4N5N6_BINS} | Time ={N1N4N5N6_TIME}")
print(f"N1,N2 and LS | Cost = {N1N2LS_COST} | Bins = {N1N2LS_BINS} | Time ={N1N2LS_TIME}")


N1 | Cost = 56910 | Bins = 517 | Time = 47.25099301338196
N1 and N4 | Cost = 50970 | Bins = 454 | Time =39.85288858413696
N1 and LS | Cost = 50870 | Bins = 460 | Time =16.604002952575684
N1,N2 and LS | Cost = 50810 | Bins = 427 | Time =24.185479640960693


In [21]:
# sys.stdout = old_stdout

# log_file.close()
